In [ ]:
import pandas as pd
import numpy as np

In [ ]:
url = "https://raw.githubusercontent.com/fahmiagungmaulana/data1/main/train.csv"
train = pd.read_csv(url)

In [ ]:
train.head()

,waktu_setempat,id_jalan,id_titik_mulai,id_titik_akhir,rerata_kecepatan
0,2020-02-01 01:00:00+00:00,691007296,21390008,1425033102,29.126
1,2020-02-01 01:00:00+00:00,47010584,1677092762,579493410,46.576
2,2020-02-01 01:00:00+00:00,22932408,26486694,1930267566,36.587
3,2020-02-01 01:00:00+00:00,142479648,1111592522,3775231113,34.063
4,2020-02-01 01:00:00+00:00,8504977,5940503398,5940503394,38.336


In [ ]:
train['kelompok'] = train.apply(lambda row: '_'.join(sorted([str(row['id_titik_mulai']), str(row['id_titik_akhir'])])), axis=1)

In [ ]:
train.kelompok.nunique()

467

In [ ]:
data_kel = train.kelompok.unique()

In [ ]:
# Inisialisasi list untuk menyimpan angka bagian kiri dan kanan
angka_kiri = []
angka_kanan = []

# Iterasi melalui setiap kelompok dan memisahkan angka
for kelompok in data_kel:
    angka = kelompok.split("_")
    angka_kiri.append(int(angka[0]))
    angka_kanan.append(int(angka[1]))

# Membuat dataframe baru
data = {'Angka_Kiri': angka_kiri, 'Angka_Kanan': angka_kanan}
df = pd.DataFrame(data)


In [ ]:
train_mulai = pd.read_csv("train_mulai.csv")

In [ ]:
# Merge dataframe df dengan train_mulai berdasarkan angka_kiri dan node_id
df_mulai = df.merge(train_mulai, left_on='Angka_Kiri', right_on='node_id', how='left')

# Menghapus kolom node_id agar tidak ada duplikasi
df_mulai = df_mulai.drop(columns='node_id')

In [ ]:
df_mulai = df_mulai.rename(columns={'latitude':'lat1','longitude':'long1'})

In [ ]:
train_akhir = pd.read_csv("train_akhir.csv")

In [ ]:
# Merge dataframe df dengan train_mulai berdasarkan angka_kiri dan node_id
df_full = df_mulai.merge(train_akhir, left_on='Angka_Kanan', right_on='node_id', how='left')

# Menghapus kolom node_id agar tidak ada duplikasi
df_full = df_full.drop(columns='node_id')

In [ ]:
df_full = df_full.rename(columns={'latitude':'lat2','longitude':'long2'})

In [ ]:
from geopy.distance import distance
# Fungsi untuk menghitung jarak antara dua koordinat
def hitung_jarak(row):
    koordinat1 = (row['lat1'], row['long1'])
    koordinat2 = (row['lat2'], row['long2'])
    return distance(koordinat1, koordinat2).kilometers

# Menambahkan kolom jarak ke DataFrame
df_full['jarak_km'] = df_full.apply(hitung_jarak, axis=1)

In [ ]:
df_full.head()

,Angka_Kiri,Angka_Kanan,lat1,long1,lat2,long2,jarak_km
0,1425033102,21390008,51.434975,-0.161090,51.434928,-0.161176,0.007931
1,1677092762,579493410,51.623990,-0.176398,51.623811,-0.176424,0.019997
2,1930267566,26486694,51.452749,-0.151896,51.452530,-0.152437,0.044792
3,1111592522,3775231113,51.465851,-0.154188,51.465654,-0.155334,0.082587
4,5940503394,5940503398,51.530197,-0.227949,51.530289,-0.228343,0.029201


In [ ]:
df_full.to_csv("Jarak.csv", index=False)